In [0]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [0]:
seq_len = 30
feature_size = 2
batch_size = 16
num_generation_steps = 30

np_data = np.random.normal(0, 1, (batch_size, seq_len, feature_size))
zero_step = np.zeros((batch_size, 1, feature_size))
tf_input = tf.Variable(np_data, dtype=tf.float32)
tf_zero_step = tf.Variable(zero_step, dtype=tf.float32)

In [0]:
node_encoder = tf.keras.Sequential()
node_encoder.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
node_encoder.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,)))
node_encoder.add(tf.keras.layers.Dense(64, activation='relu'))

node_decoder_inp = tf.keras.layers.Dense(64, activation='relu')
node_decoder_lstmcell = tf.keras.layers.LSTMCell(64)
node_decoder_rnn = tf.keras.layers.RNN(node_decoder_lstmcell, return_sequences=True)
node_decoder_out = tf.keras.layers.Dense(feature_size, activation='linear')

node_decoder = tf.keras.Sequential([node_decoder_inp, node_decoder_rnn, node_decoder_out])

In [0]:
node_embedding = node_encoder(tf_input)
node_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             multiple                  192       
_________________________________________________________________
bidirectional_4 (Bidirection multiple                  66048     
_________________________________________________________________
dense_27 (Dense)             multiple                  8256      
Total params: 74,496
Trainable params: 74,496
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Decoder at traning time. I.e., we always feed the previous ground-truth step.
# We feed the node embedding at every time-step.

decoder_inputs = tf.concat([tf.tile(tf.expand_dims(node_embedding, axis=1), (1, seq_len, 1)), tf_input], axis=-1)
print(decoder_inputs.shape)

reconstructed = node_decoder(decoder_inputs)
print(reconstructed.shape)

(16, 30, 66)


AttributeError: ignored

In [0]:
# Decoder at sampling time. Generates a sequence sample auto-regressively.

generated = []
current_input = tf.concat([node_embedding, tf_zero_step[:, -1, :]], axis=-1)
current_state=[tf.Variable(np.zeros((batch_size, 64)), dtype=tf.float32), tf.Variable(np.zeros((batch_size, 64)), dtype=tf.float32)]

for i in range(num_generation_steps):
  decoder_inp_forward = node_decoder_inp(current_input)
  [decoder_rnn_forward, current_state] = node_decoder_lstmcell(decoder_inp_forward, states=current_state)
  prediction = node_decoder_out(decoder_rnn_forward)
  current_input = tf.concat([node_embedding, prediction], axis=-1)
  generated.append(prediction)

(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)
(16, 2)


TensorShape([Dimension(16), Dimension(2)])